In [34]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
ss = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

In [35]:
import pandas as pd
import glob

path = 'OutputData/'
all_files = glob.glob(path + "/*.csv")
big_csv = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    big_csv.append(df)

frame = pd.concat(big_csv, axis=0, ignore_index=True)

In [36]:
frame.drop(['Unnamed: 0'],axis=1,inplace=True)
frame.to_csv('final_chunk.csv', index=False)

In [39]:
# read the file with features
features = ss.read.csv('final_chunk.csv', header=True)
# read the file of speaker id and gender
gender = ss.read.csv('Lab41-SRI-VOiCES-speaker-gender-dataset.csv', header=True)

In [40]:
features.show(4)

+--------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-----------------+-------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+--------------------+
|            FileName|          Centroid|          variance|          skewness|          kurtosis|              mfcc1|             mfcc2|              mfcc3|            mfcc4|              mfcc5|            mfcc6|              mfcc7|              mfcc8|              mfcc9|             mfcc10|             mfcc11|             mfcc12|      roll_off_max|      roll_off_min|            flatness|   zeroCrossingRate|                 rms|
+--------------------+------------------+------------------+------------------+------------------+-------------------+--------------

In [41]:
# define a udf that extracts the speaker id out of the filename
def get_speaker(filename):
    i = filename.index('sp')
    return int(filename[i+2:i+6])
speaker = udf(get_speaker)

In [42]:
# process speaker id
features_df = features.withColumn('Speaker', speaker('FileName')).drop('FileName', '_c0')

In [43]:
# join the dataframe
results = features_df.join(gender, on='Speaker', how='leftOuter')

In [44]:
# check the number of observations
print(features.count())
print(results.count())

9381
9381


In [45]:
sc.stop()